# Character-level RNN From Scratch in NumPy

This project implements a simple character-level RNN trained to generate Shakespeare-like text.  
It builds the RNN completely from scratch using NumPy, trains on small sequences, and generates text.

Start by reading the Shakepeare text and cleaning it:
- Lowercasing all letters
- Replacing all non-alphabetic characters with spaces
- Removing extra whitespace

In [ ]:
import numpy as np
import re

# Load the text file and convert to lowercase
text = open("smallShake.txt", "r").read().lower()

# Keep only letters, replace everything else with a space
text = re.sub('[^A-Za-z]+', ' ', text).strip()
text

'to be or not to be that is the question whether tis nobler in the mind to suffer the slings and arrows of outrageous fortune or to take arms against a sea of troubles and by opposing end them'

Next we create a set of all the unique characters in the text. I use a set to check whether or not a character has been seen because it is more efficient then going through a list.

In [ ]:
l = []
s = set()

for i in text:
    if i not in s:
        l.append(i)
        s.add(i)

print(l)

['t', 'o', ' ', 'b', 'e', 'r', 'n', 'h', 'a', 'i', 's', 'q', 'u', 'w', 'l', 'm', 'd', 'f', 'g', 'k', 'y', 'p']


Then we create a character vocabulary mapping each unique character to an index. This is used to convert characters to one-hot encoded vectors. 

In [ ]:
char_to_index = {}
index_to_char = {}
for i, ch in enumerate(l):
    char_to_index[ch] = i
    index_to_char[i] = ch

print(char_to_index)
print(index_to_char)

{'t': 0, 'o': 1, ' ': 2, 'b': 3, 'e': 4, 'r': 5, 'n': 6, 'h': 7, 'a': 8, 'i': 9, 's': 10, 'q': 11, 'u': 12, 'w': 13, 'l': 14, 'm': 15, 'd': 16, 'f': 17, 'g': 18, 'k': 19, 'y': 20, 'p': 21}
{0: 't', 1: 'o', 2: ' ', 3: 'b', 4: 'e', 5: 'r', 6: 'n', 7: 'h', 8: 'a', 9: 'i', 10: 's', 11: 'q', 12: 'u', 13: 'w', 14: 'l', 15: 'm', 16: 'd', 17: 'f', 18: 'g', 19: 'k', 20: 'y', 21: 'p'}


Now we convert the entire text into:
- A list of token indices, each representing a character.
- A one-hot encoded matrix, where each row is a one-hot vector representing a character.

In [ ]:
tokens = []
for ch in text:
    tokens.append(char_to_index[ch])

# Create one-hot encoded sparse matrix
sparse = np.zeros(shape=(len(tokens), len(l)))
for i in range(len(tokens)):
    sparse[i, tokens[i]] = 1

print(sparse[0:2]) # Should be in the first and second position

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


# Utility Functions

In [538]:
def tanh(x):
    """
    Element-wise hyperbolic tangent (tanh) activation function.

    Parameters:
    - x: np.ndarray or float
        Input value(s).

    Returns:
    - np.ndarray or float
        Output after applying the tanh function element-wise.
    """
    return np.tanh(x)

def tanh_deriv(x):
    """
    Derivative of the hyperbolic tangent (tanh) function.

    Parameters:
    - x: np.ndarray or float
        Input value(s).

    Returns:
    - np.ndarray or float
        The derivative of tanh, computed as 1 - tanh(x)^2, evaluated at x.
    """
    return 1.0 - np.tanh(x)**2

def softmax(Z: np.ndarray) -> np.ndarray:
    """
    Compute softmax probabilities over classes.

    Parameters:
    - Z: np.ndarray
        Logits of shape (batch_size, num_classes).

    Returns:
    - np.ndarray
        Softmax probabilities of shape (batch_size, num_classes).
    """
    Z = Z - np.max(Z, axis=1, keepdims=True)
    exp_Z = np.exp(Z)
    return exp_Z / np.sum(exp_Z, axis=1, keepdims=True)

def CrossEntropy(yhat: np.ndarray, y: np.ndarray, eps: float = 1e-15) -> float:
    """
    Mean cross-entropy loss for multi-class classification.

    Parameters:
    - yhat: np.ndarray
        Predicted probabilities (batch_size, num_classes).
    - y: np.ndarray
        One-hot encoded true labels (batch_size, num_classes).
    - eps: float, optional
        Small epsilon to avoid log(0).

    Returns:
    - float
        Mean cross-entropy loss over the batch.
    """
    yhat = np.clip(yhat, eps, 1 - eps)
    return -np.mean(np.sum(y * np.log(yhat), axis=1))

# Preparing the training sequences

To train the RNN, we need to create input-output pairs of sequences.
Each input sequence (X) consists of 20 consecutive characters, represented as one-hot vectors.
The target (y) is the next sequence shifted by one character, so the RNN learns to predict the next character at each time step.

In [ ]:
X = np.array([sparse[x:x+20] for x in range(len(sparse) - 20)])
y = np.array([sparse[x+1:x+21] for x in range(len(sparse) - 20)])

print("X shape:", X.shape)  # (num_sequences, sequence_length, vocab_size)
print("y shape:", y.shape)  # (num_sequences, sequence_length, vocab_size)

X shape: (171, 20, 22)
y shape: (171, 20, 22)


## Initializing the RNN weights

- Input-to-hidden weights `wx`
- Hidden-to-hidden weights `wh`
- Hidden bias `b`
- Hidden-to-output weights `wy`
- Output bias `by`

We also configure hyperparameters:
- `hidden_size` = number of hidden units
- `sequence_length` = length of input sequences
- `lr` = learning rate
- `epochs` = maximum number of epochs to train


In [566]:
hidden_size = 64
output_size = len(l)   # Vocab size
input_size = 22        # One-hot size
sequence_length = 20   # Number of time steps to unroll

# xt = X[i, t, :]                                    # (22,)
wx = np.random.rand(hidden_size, input_size) *0.01   # (64, 22)
wh = np.random.rand(hidden_size, hidden_size) * 0.01 # (64, 64)
b = np.zeros(shape=hidden_size)                      # (64,)

wy = np.random.rand(output_size, hidden_size) * 0.01 # (22, 64)
by = np.zeros(shape=output_size)                     # (22,)

lr = 0.0005
epochs = 500
past_loss = []
break_loop = False

## Training loop

We iterate over each epoch and each training sequence.

For each sequence:
- Forward pass through time to compute hidden states and outputs.
- Compute cross-entropy loss and gradients.
- Then do backpropagation through time to compute gradients.
- Use gradient clipping to prevent exploding gradients.
- Finally, update all weights.

Early stopping stops training if the loss stops improving.


In [567]:
for epoch in range(epochs):
    total_loss = 0

    for i in range(len(X)):
        # Reset hidden state for each new sequence
        h = np.zeros(shape=(1, hidden_size))   # (1, 64)

        logits = []
        h_states = []

        # Forward Pass
        for t in range(sequence_length):
            xt = X[i, t, :]
            h = tanh(xt @ wx.T + h @ wh.T + b)  # (1, 64)
            logits.append(h @ wy.T + by)        # (1, 22)
            h_states.append(h.copy())

        y_pred = np.stack(logits, axis=0).squeeze(1)      # (20, 22)
        h_states = np.stack(h_states, axis=0).squeeze(1)  # shape (20, 64)

        # Loss
        probs = softmax(y_pred)                 # (20, 22)
        loss = CrossEntropy(probs, y[i])
        total_loss += loss

        # Backpropagation
        dz_out = probs - y[i]                   # (20, 22)
        dw_out = np.dot(dz_out.T, h_states)     # (22, 20) x (20, 64) --> (22, 64)
        db_out = np.sum(dz_out, axis=0)         # (22,)

        dh = np.dot(dz_out, wy)                 # (20, 64)

        dwx = np.zeros(shape=(hidden_size,22))
        dwh = np.zeros(shape=(hidden_size,hidden_size))
        db = np.zeros(shape=(hidden_size,))
        
        dh_next = np.zeros(shape=(hidden_size))
        for t in reversed(range(sequence_length)):
            
            dh_total = dh[t] + dh_next   # (64,)

            dtanh = dh_total * tanh_deriv(h_states[t])  # shape (64,)

            dwx += np.outer(dtanh, X[i, t, :])          # (64,22)
            db += dtanh                                 # (64,)

            if t > 0:
                dwh += np.outer(dtanh, h_states[t-1])  # (64,64)
            else:
                dwh += np.outer(dtanh, np.zeros_like(h_states[0]))

            dh_next = np.dot(dtanh, wh)  # shape (64,)
        
        # Gradient Clipping
        np.clip(dwx, -1, 1, out=dwx)
        np.clip(dwh, -1, 1, out=dwh)
        np.clip(db, -1, 1, out=db)
        np.clip(dw_out, -1, 1, out=dw_out)
        np.clip(db_out, -1, 1, out=db_out)

        # Update weights
        wx -= lr * dwx
        wh -= lr * dwh
        b  -= lr * db
        wy -= lr * dw_out
        by -= lr * db_out

    # Early stopping and displaying loss
    avg_loss = total_loss / len(X)
    print(f"Epoch {epoch+1}, loss: {avg_loss:.4f}")

    if len(past_loss) > 5:
        if past_loss[-1] < avg_loss or past_loss[-2] < avg_loss:
            break_loop = True
    past_loss.append(avg_loss)

    if break_loop:
        break
    

Epoch 1, loss: 3.0723
Epoch 2, loss: 3.0350
Epoch 3, loss: 3.0006
Epoch 4, loss: 2.9686
Epoch 5, loss: 2.9378
Epoch 6, loss: 2.9066
Epoch 7, loss: 2.8718
Epoch 8, loss: 2.8279
Epoch 9, loss: 2.7754
Epoch 10, loss: 2.7421
Epoch 11, loss: 2.7269
Epoch 12, loss: 2.7200
Epoch 13, loss: 2.7164
Epoch 14, loss: 2.7141
Epoch 15, loss: 2.7123
Epoch 16, loss: 2.7108
Epoch 17, loss: 2.7094
Epoch 18, loss: 2.7080
Epoch 19, loss: 2.7067
Epoch 20, loss: 2.7053
Epoch 21, loss: 2.7038
Epoch 22, loss: 2.7021
Epoch 23, loss: 2.7003
Epoch 24, loss: 2.6982
Epoch 25, loss: 2.6958
Epoch 26, loss: 2.6930
Epoch 27, loss: 2.6897
Epoch 28, loss: 2.6858
Epoch 29, loss: 2.6812
Epoch 30, loss: 2.6757
Epoch 31, loss: 2.6693
Epoch 32, loss: 2.6617
Epoch 33, loss: 2.6531
Epoch 34, loss: 2.6432
Epoch 35, loss: 2.6323
Epoch 36, loss: 2.6203
Epoch 37, loss: 2.6075
Epoch 38, loss: 2.5939
Epoch 39, loss: 2.5797
Epoch 40, loss: 2.5649
Epoch 41, loss: 2.5493
Epoch 42, loss: 2.5329
Epoch 43, loss: 2.5154
Epoch 44, loss: 2.49

## Softmax with temperature

We use a softmax function with a temperature parameter:
- Lower temperature (<1) makes the output distribution sharper, favoring the most likely next characters.
- Higher temperature (>1) makes it more random.

Temperature helps tune how "creative" vs "predictable" the generated text will be.


In [543]:
def softmax_temp(logits, temp=0.6):
    logits = logits / temp
    e_logits = np.exp(logits - np.max(logits))
    return e_logits / np.sum(e_logits)

## Generation function

This function takes a starting string, feeds it through the trained RNN to warm up the hidden state, then generates new characters one by one, feeding each predicted character back into the model.

In [ ]:
def generate(start_seq, length=100):

    # Clean the text
    start_seq = re.sub('[^A-Za-z]+', ' ', start_seq).lower().strip()
    gen_tokens = []
    for ch in start_seq:
        gen_tokens.append(char_to_index[ch])

    # Convert to one-hot vectors
    gen_sparse = np.zeros(shape=(len(gen_tokens), len(l)))
    for i in range(len(gen_tokens)):
        gen_sparse[i, gen_tokens[i]] = 1
    
    # Initialize hidden state
    h = np.zeros(hidden_size)

    # Feed starting sequence
    for xt in gen_sparse:
        h = tanh(xt @ wx.T + h @ wh.T + b)
    
    # Start generation from the last character
    current_input = gen_sparse[-1]
    generated = []

    # Generate new characters
    for i in range(length):
        h = tanh(current_input @ wx.T + h @ wh.T + b)
        logits = h @ wy.T + by
        probs = softmax_temp(logits)

        next_idx = np.random.choice(len(probs), p=probs)
        next_onehot = np.zeros_like(current_input)
        next_onehot[next_idx] = 1

        generated.append(next_idx)
        current_input = next_onehot

    # Decode back to string
    gen_text = ""
    for x in generated:
        gen_text += index_to_char[x]

    return gen_text


Generating Text

In [547]:
gen_text = generate("To be, or not to be, that is the question:")
print(gen_text)

whether tis nobler in the mind to sunfer the slings and arrows of outrageous fortune or the mirst on


# Conclusion

There is an occasional error like "sunfer", but otherwise it generates well. With more data or an LSTM, it would learn and generate better.

This project demonstrates how to build a working character-level RNN entirely from scratch, understanding every detail of:
- How sequences flow through time via the hidden state.
- How backpropagation works across multiple time steps.
- How sampling with softmax temperature can control the creativity of generated text.

There were a few problems encountered along the way. At first I didn't clip the gradients so they exploded very fast. The second was that I initialized the hidden state (h) outside of the training loop. This made it so every single time a new sequence came in, h wasn't reset, so it would have a bunch of garbage values leftover from the last sequence, lowering the performance of the model. This caused the model to predict random characters. When I fixed the problem and the hidden state was reset after every sequence, performance went up and generated coherent words.